In [ ]:
# stdlib
import os

# syft absolute
import syft as sy
# from syft import test_settings

# syft absolute
from syft.service.worker.image_registry import SyftImageRegistry
from syft.service.worker.worker_image import SyftWorkerImage

In [ ]:
os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
os.environ["DEV_MODE"] = "True"

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    # reset=True,
    # port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

In [ ]:
high_client.worker_pools

In [ ]:
assert len(high_client.worker_pools.get_all()) == 1

# Local registry and image stuff

In [ ]:
from syft.util.util import get_latest_tag

registry = os.getenv("SYFT_BASE_IMAGE_REGISTRY", "k3d-registry.localhost:5800")
repo = "openmined/syft-backend"

if "k3d" in registry:
    tag = get_latest_tag(registry, repo)
else:
    tag = sy.__version__

print(f"registry: {registry}")
print(f"repo: {repo}")
print(f"tag: {tag}")

In [ ]:
bigquery_dockerfile_str = f"""
FROM {registry}/{repo}:{tag}

RUN uv pip install google-cloud-bigquery db-dtypes

""".strip()

In [ ]:
docker_config = sy.DockerWorkerConfig(dockerfile=bigquery_dockerfile_str)
assert docker_config.dockerfile == bigquery_dockerfile_str

In [ ]:
submit_result = high_client.api.services.worker_image.submit(
    worker_config=docker_config
)
submit_result

In [ ]:
assert isinstance(submit_result, sy.SyftSuccess), str(submit_result)

In [ ]:
dockerfile_list = high_client.images.get_all()
dockerfile_list

In [ ]:
workerimage = next(
    (
        image
        for image in dockerfile_list
        if not image.is_prebuilt and image.config.dockerfile == bigquery_dockerfile_str
    ),
    None,
)

assert isinstance(workerimage, SyftWorkerImage), str(workerimage)
workerimage

In [ ]:
external_registry = os.getenv("EXTERNAL_REGISTRY", registry)
external_registry_username = os.getenv("EXTERNAL_REGISTRY_USERNAME", None)
external_registry_password = os.getenv("EXTERNAL_REGISTRY_PASSWORD", None)

In [ ]:
registry_add_result = high_client.api.services.image_registry.add(external_registry)
registry_add_result

In [ ]:
image_registry_list = high_client.api.services.image_registry.get_all()
image_registry_list

In [ ]:
local_registry = image_registry_list[0]
local_registry

# Build Image

In [ ]:
docker_tag = "openmined/bigquery:0.9.1-beta.6"


docker_build_result = high_client.api.services.worker_image.build(
    image_uid=workerimage.id,
    tag=docker_tag,
    registry_uid=local_registry.id,
)
docker_build_result

In [ ]:
assert not isinstance(docker_build_result, sy.SyftError), str(docker_build_result)

In [ ]:
image_list = high_client.images.get_all()
image_list

# Push to local registry and launch pool

In [ ]:
push_result = None
push_result = high_client.api.services.worker_image.push(
    workerimage.id,
    username=external_registry_username,
    password=external_registry_password,
)
push_result

In [ ]:
worker_pool_name = "bigquerypool"
custom_pool_pod_annotations = {"bigquery-custom-pool": "Pod annotation for bigquery"}
custom_pool_pod_labels = {"bigquery-custom-pool": "Pod_label_for_bigquery"}

In [ ]:
result = high_client.api.services.worker_pool.launch(
    pool_name=worker_pool_name,
    image_uid=workerimage.id,
    num_workers=1,
    registry_username=external_registry_username,
    registry_password=external_registry_password,
    pod_annotations=custom_pool_pod_annotations,
    pod_labels=custom_pool_pod_labels,
)
result

In [ ]:
assert len(high_client.worker_pools.get_all()) == 2

In [ ]:
base_worker_image = high_client.images.get_all()[0]
base_worker_image

# Rest of the stuff

In [ ]:
high_client.register(
    email="data_scientist@openmined.org",
    password="verysecurepassword",
    password_verify="verysecurepassword",
    name="John Doe",
)

In [ ]:
high_client.settings.allow_guest_signup(enable=False)

In [ ]:
assert len(high_client.api.services.user.get_all()) == 2